In [0]:
%run /Users/pnl0rc8b@emea.royalahold.net/FINALIZED_Notebooks/Functions_Finalized

In [0]:
%run /Users/pnl0rc8b@emea.royalahold.net/FINALIZED_Notebooks/Functions_Tables

#Configureer Parameters

In [0]:
# PARAMETERS
ATTRIBUUT = "ndContainsCocos" # Selecteer Attribuut

In [0]:
numerieke_features = load_numerical_features(ATTRIBUUT)

# Load model from database

In [0]:
#dbutils.fs.cp("dbfs:/FileStore/nd_MODELS_publicatie/model_"+str(ATTRIBUUT)+".h5", "file:/FileStore/nd_MODELS_publicatie/model_"+str(ATTRIBUUT)+"_1__LOADED.h5")
from tensorflow import keras
model = keras.models.load_model("/dbfs/FileStore/nd_MODELS/model_"+str(ATTRIBUUT)+".h5")

# Formateer Attribuut

In [0]:
# onderstaande twee parameters, hoeven niet aangepast te worden.
Classes_threshold = 11
Overig_threshold = 150 # Class beneden deze threshold, zal worden toegevoegd aan class: 'overig'

nd_controle = koppel_ndAttributen_new_products(NASA_CONTROLE)
df_geformateerd, multi_label_flag, classes_available = create_overige_class(nd_controle, ATTRIBUUT,Classes_threshold, Overig_threshold)

DF_merged = koppel_preprocessed_text(df_geformateerd, DF_NLP_spark) # Hier moeten we het nieuwe assortiment door de tokenizer laten loopen
DF_merged = fix_empty_fields(DF_merged, ATTRIBUUT)
DF_merged = Correlation_matrix(DF_merged, ATTRIBUUT, show_nan_plot=False, show_correlation_matrix=False)

# Neural network
numerical, text, target, vocab_size = single_prep_controle(numerieke_features, DF_merged, ATTRIBUUT) # Preparation, before training

#Predictions
list_ytest, list_yhat, yhat2 = predictions_2inputs(model, numerical, text, target)
plot_confusion_matrix(DF_merged, confusion_matrix(list_ytest, list_yhat), ATTRIBUUT)
accuracy_report(list_ytest, list_yhat, macro_f1_list=[])
#Predictions to dataframe
controle_tabel = predictions_to_dataframe(DF_merged, ATTRIBUUT, list_ytest, list_yhat)

# Controle Tabellen

In [0]:
equal, unequal = unequal_and_equal_table(controle_tabel, ATTRIBUUT) # Both spark dataframes

In [0]:
display(unequal) # Met deze tabel, kan je inzien welke producten er nog moeten worden gecontroleerd. Dus hier is busines-rules ongelijk aan de voorspellingen van het machine learning model.

In [0]:
display(equal)